In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import pandas as pd
import pprint

from privex.components.basic import Schema, Dataset, GroupbyQuery, Question
from privex.components.utils import generate_explanation_predicates
from privex.framework.solution import ExplanationSession

import logging
logger = logging.getLogger(__name__)

In [5]:
df = pd.read_csv('../data/german/synthetic_german.csv')
print(len(df))
schema = Schema.from_json('../data/german/german.json')
dataset = Dataset(df, schema)
gamma = 0.95
attributes = ['duration', 'credit-history', 'purpose', 'credit-amount', 'saving-account', 'employment', 'installment-rate', 'sex-marst', 'other-debtors', 'residence', 'property', 'age', 'other-installment-plans', 'housing', 'existing-credits', 'job', 'people-liable', 'telephone', 'foreign-worker']
predicates = generate_explanation_predicates(attributes, schema, strategy='1-way marginal')
#predicates = predicates[:10]
#predicates += generate_explanation_predicates(attributes, schema, strategy='2-way marginal')
es = ExplanationSession(dataset, gamma, predicates, random_seed = 152636)

1000000
2022-04-18 16:00:39,198 INFO     [meta_explanation_session.py:23] 82 predicates for the explanation.


In [7]:
# Phase 1
groupby_query = GroupbyQuery(
    agg = 'AVG',
    attr_agg = 'good-credit',
    predicate = None,
    attr_group = 'status',
    schema = schema
)
rho_query = 0.1
es.phase_1_submit_query(groupby_query, rho_query)
print(f'submiited queries with rho = {rho_query}')
nr = es.phase_1_show_query_results()
nr['group'] = nr['group'].apply(lambda row: row[0])
gt = df.groupby('status').agg(answer=('good-credit', 'mean')).reset_index()
gt = gt.rename(columns={'status': 'group'})
print(nr.merge(gt, on='group').rename(columns={'answer_x':'answer', 'answer_y': 'truth (hidden)'}).sort_values('truth (hidden)'))

submiited queries with rho = 0.1
                                        group    answer  truth (hidden)
0                         no checking account  0.526571        0.526574
1                                  ... < 0 DM  0.574447        0.574466
2                            0<= ... < 200 DM  0.756406        0.756344
3  ... >= 200 DM / salary for at least 1 year  0.863100        0.863093


In [8]:
# Phase 2
question = Question.from_group_comparison(groupby_query, ('... < 0 DM',), ('no checking account',))
es.phase_2_submit_question(question)
es.phase_2_prepare_question_ci()
ci = es.phase_2_show_question_ci()
print('question: ', question.to_natural_language())
print(f'The {gamma*100:.0f}% confidence interval of the difference is ', ci)
logger.info(f'question_point: {es.question_point}')
groundtruth_ci = es.phase_2_ground_truth_ci(rho_query, gamma)
logger.info(f'true question: {es.question.evaluation(es.dataset)}')
print(f'The {gamma*100:.0f}% confidence interval of ground truth is ', groundtruth_ci)

question:  Why AVG(good-credit) WHERE `status` == "... < 0 DM" >= AVG(good-credit) WHERE `status` == "no checking account"?
The 95% confidence interval of the difference is  (0.04778608102170201, 0.0479666572405788)
2022-04-18 16:02:10,633 INFO     [<ipython-input-8-4c19dd358107>:8] question_point: 0.04787636904818948
2022-04-18 16:02:11,191 INFO     [<ipython-input-8-4c19dd358107>:10] true question: 0.04789193497607158
The 95% confidence interval of ground truth is  (0.04785552724148564, 0.047928231271848365)


In [13]:
# Phase 3
k = 5
logger.debug(f'Length of predicates is {len(predicates)}')
es.phase_3_submit_explanation_request()
es.phase_3_prepare_explanation(k, 
    {'rho_topk': 0.5, 'rho_influ': 0.5, 'rho_rank': 1.0}, split_factor = 0.9)
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
'display.max_colwidth', 1000, 'display.width', 1000):
    t1, t2 = es.phase_3_show_explanation_table()
    print(t1)
    print(t2)

2022-05-07 22:34:48,72 DEBUG    [<ipython-input-13-716482da31c9>:3] Length of predicates is 82
(`status` == "no checking account")or(`status` == "... < 0 DM")
2022-05-07 22:34:48,193 INFO     [influence_function.py:24] Dataset relative to the question has length 542650
2022-05-07 22:35:27,953 DEBUG    [meta_explanation_session.py:169] 82 predicates and their influences & scores have been loaded.
2022-05-07 22:35:27,960 DEBUG    [meta_explanation_session.py:173] 
       influence         score
count  82.000000     82.000000
mean    0.000333     91.117826
std     0.014304   3912.349348
min    -0.057912 -15839.828271
25%    -0.004585  -1253.962692
50%     0.000003      0.761900
75%     0.005414   1480.750210
max     0.036828  10073.068379
2022-05-07 22:35:27,961 DEBUG    [meta_explanation_session.py:213] total rho_expl is 2.0
2022-05-07 22:35:27,961 DEBUG    [meta_explanation_session.py:214] rho_topk is 0.5
2022-05-07 22:35:27,962 DEBUG    [meta_explanation_session.py:215] rho_ci is 0.5
2

100%|██████████| 5/5 [00:02<00:00,  2.41it/s]

2022-05-07 22:35:30,46 DEBUG    [meta_explanation_session.py:259] [(9975.491082205894, 10115.734688698145), (9155.667088558615, 9295.910695050867), (7055.036963808659, 7195.280570300912), (6374.157084359778, 6514.400690852031), (6246.487777454008, 6386.731383946261)]
2022-05-07 22:35:30,46 INFO     [meta_explanation_session.py:267] computing rank ci



100%|██████████| 5/5 [00:00<00:00, 4028.34it/s]

2022-05-07 22:35:30,50 DEBUG    [meta_explanation_session.py:280] [(1, 1), (1, 2), (2, 4), (2, 5), (4, 5)]
                                                          predicates Rel Inf 90-CI L Rel Inf 90-CI R  Rnk 95-CI L  Rnk 95-CI R
0                                          `existing-credits` == "1"          76.18%          77.25%            1            1
1                             `job` == "skilled employee / official"          69.92%          70.99%            1            2
2                            `sex-marst` == "male : married/widowed"          53.88%          54.95%            2            4
3                                   `credit-amount` == "(500, 2500]"          48.68%          49.75%            2            5
4  `credit-history` == "no credits taken/all credits paid back duly"          47.70%          48.77%            4            5
                                                          predicates  Inf 95-CI L   Inf 95-CI R  Rnk 95-CI L  Rnk 95-CI R
0        

In [10]:
def get_influence_at_rank_i(es, i):
    return es.predicates_with_influences[
        es.sorted_predicates[i-1]
    ]

def get_score_at_rank_i(es, i):
    return es.predicates_with_scores[
        es.sorted_predicates[i-1]
    ]

In [11]:
true_kth_influence = get_influence_at_rank_i(es, k)
logger.info(f'topk true top_k influence: {[get_influence_at_rank_i(es, i+1) for i in range(k)]}')
logger.info(f'topk true top_k score: {[get_score_at_rank_i(es, i+1) for i in range(k)]}')

2022-04-18 16:13:51,613 INFO     [<ipython-input-11-8a65cf92c78b>:2] topk true top_k influence: [0.036828077256122645, 0.03384325947850492, 0.02596021355695329, 0.023686919061497044, 0.02315438433232411]
2022-04-18 16:13:51,614 INFO     [<ipython-input-11-8a65cf92c78b>:3] topk true top_k score: [10073.068378785641, 9256.672959522752, 7100.533771243636, 6478.751354024425, 6333.094585039961]


In [15]:
k = 5
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
'display.max_colwidth', 1000, 'display.width', 1000):
    print(es.phase_3_true_top_k(k))

Positive Influences:  38
                                                                topk rel-influence
0                                          `existing-credits` == "1"        76.92%
1                             `job` == "skilled employee / official"        70.69%
2                            `sex-marst` == "male : married/widowed"        54.22%
3                                   `credit-amount` == "(500, 2500]"        49.47%
4  `credit-history` == "no credits taken/all credits paid back duly"        48.36%
